## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [1]:
%matplotlib qt

from helper import *

#Calibrating camera
ret, mtx, dist, rvecs, tvecs = calibration()

In [187]:
#Pipeline for Lane Detection
file = 'test_images/test1.jpg'
original = cv2.imread(file)
#Undistort Image
undist = cv2.undistort(original, mtx, dist, None, mtx)
#Create Threshold
hls_img = hls_select(undist, thresh=(100,255))
mag_img = mag_thresh(undist, mag_thresh=(50, 255))
dir_img = dir_threshold(undist, thresh=(np.pi/4,2*np.pi/4))
sobelx_img = abs_sobel_thresh(undist, orient='x', thresh_min=50, thresh_max=100)
sobely_img = abs_sobel_thresh(undist, orient='y', thresh_min=50, thresh_max=100)

#Combine Threshold
combined = np.zeros_like(dir_img)
combined[((sobelx_img == 255) & (sobely_img == 255)) | ((mag_img == 255) & (dir_img == 255)) | (hls_img == 255)] = 255


In [236]:
# Tuning Warp Images
straight = cv2.imread('test_images/straight_lines2.jpg')
# Pts for polylines use int32 type
src2 = np.int32([[xsize//2 - 50, 450 ],
                  [xsize//2 + 53, 450],
                  [xsize//2 + 465,ysize-10],
                  [xsize//2 - 430,ysize-10]])
cv2.polylines(straight, [src2], True, (0,0,255), 3)
cv2.imshow("Straight",straight)

In [188]:
#Warp Image
xsize, ysize = (combined.shape[1], combined.shape[0])
# Pts for warpPerspectiveTransform use float32 type
src = np.float32([[xsize//2 - 50, 450 ],
                  [xsize//2 + 53, 450],
                  [xsize//2 + 465,ysize-10],
                  [xsize//2 - 430,ysize-10]])
dst = np.float32([[300, 0],
                 [1000, 0],
                 [1000, 720],
                 [300, 720]])
# M = cv2.getPerspectiveTransform(src, dst)
# warped = cv2.warpPerspective(combined, M, img_size)
warped = warper(combined, src, dst)

In [180]:
cv2.imshow("ori",original)
cv2.imshow("comb",combined)
plt.imshow(warped)

In [231]:
out_img, left_fit, right_fit = fit_polynomial(warped)

In [232]:
out_img_search = search_around_poly(warped)

Left
[False False False ... False False False]
Right
(86204,)


In [235]:
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        ### TO-DO: Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        ### Identify the nonzero pixels in x and y within the window ###
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        ### If you found > minpix pixels, recenter next window ###
        ### (`right` or `leftx_current`) on their mean position ###
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds])) # Remove this when you add your function

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img

In [234]:
def fit_polynomial(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    ### Fit a second order polynomial to each using `np.polyfit` ###
    # For Pixel
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # For Actual Road, scale it with ym and xm per pixel
    left_line.current_fit = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_line.current_fit = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines for pixel plotting
    plt.imshow(out_img)
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    plt.xlim(0, 1280)
    plt.ylim(720, 0)

    return out_img, left_fit, right_fit

In [223]:
leftx*xm_per_pix

array([1.71257143, 1.71785714, 1.72314286, ..., 2.08257143, 2.08785714,
       2.09314286])

In [224]:
leftx

array([324, 325, 326, ..., 394, 395, 396], dtype=int64)

In [22]:
plt.imshow(out_img)
# plt.plot(left_fitx, ploty, color='yellow')
# plt.plot(right_fitx, ploty, color='yellow')

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [9]:
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in meter
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None  

In [242]:
left_line = Line()
right_line = Line()

In [249]:
left_line.current_fit[1]

-0.03438927362137625

In [243]:
left_fit

array([ 3.81786898e-04, -3.43892736e-02,  2.42476305e+00])

In [226]:
def fit_poly(img_shape, leftx, lefty, rightx, righty):
    ### Fit a second order polynomial to each with np.polyfit() ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    left_line.current_fit = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_line.current_fit = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fitx, right_fitx, ploty

In [227]:
def search_around_poly(binary_warped):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### TO-DO: Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    print("Left")
    print(left_lane_inds)
    print("Right")
    print(right_lane_inds.shape)
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    # Plot the polynomial lines onto the image
#     plt.imshow(out_img)
#     plt.plot(left_fitx, ploty, color='yellow')
#     plt.plot(right_fitx, ploty, color='yellow')
#     plt.xlim(0, 1280)
#     plt.ylim(720, 0)
    ## End visualization steps ##
    
    return result, left_fitx, right_fitx

In [228]:
def measure_curvature():
    '''
    Calculates the curvature of polynomial functions in pixels.
    '''
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = 720
    
    ##### TO-DO: Implement the calculation of R_curve (radius of curvature) #####
    left_line.radius_of_curvature = ((1 + (2*left_line.current_fit[0]*y_eval*ym_per_pix + left_line.current_fit[1])**2)**1.5) / np.absolute(2*left_line.current_fit[0])
    right_line.radius_of_curvature = ((1 + (2*right_line.current_fit[0]*y_eval*ym_per_pix + right_line.current_fit[1])**2)**1.5) / np.absolute(2*right_line.current_fit[0])
    
    return left_line.radius_of_curvature, right_line.radius_of_curvature

In [233]:
measure_curvature_pixels()

(1847.441066485513, 2870.261937942105)

In [219]:
ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/700 # meters per pixel in x dimension

In [141]:
offset = 120 
xsize, ysize = (combined.shape[1], combined.shape[0])
src = np.float32([[xsize//2 - 120 , 500 ], [xsize//2 + 120,500], [xsize//2 - 430,ysize-10], [xsize//2 + 430,ysize-10]])
# dst = np.float32([[offset, img_size[1]-offset], 
#                   [img_size[0]-offset, img_size[1]-offset] ,
#                   [offset, offset], 
#                   [img_size[0]-offset, offset]])
dst = np.float32([[300, 0],
                 [1000, 0],
                 [300, 720],
                 [1000, 720]])
M = cv2.getPerspectiveTransform(src, dst)
warped = cv2.warpPerspective(combined, M, img_size)

In [118]:
xsize, ysize = (combined.shape[1], combined.shape[0])
# src = np.array([[[280,650],[500, 500 ],[770,500],  [1000,650]]], dtype=np.int32)
# src = np.array([[[500, 500 ] , [770,500], [280,650], [1000,650]]], dtype=np.int32)
dst = np.float32([[300, 0],
                 [1000, 0],
                 [300, 720],
                 [1000, 720]])
gray = cv2.cvtColor(original, cv2.COLOR_RGB2GRAY)
mask = np.zeros_like(original)
cv2.fillPoly(mask,src,0)
masked_image = cv2.bitwise_and(original, mask)

In [143]:
plt.imshow(warped)

In [124]:
plt.imshow(masked_image)

In [136]:
plt.imshow(combined)

In [46]:
cv2.imshow("combine", warped)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# img = abs_sobel_thresh(undist, orient = 'x', thresh_min = 20, thresh_max = 100)
mag_img = mag_thresh(undist, mag_thresh=(50, 255))
dir_img = dir_threshold(undist, thresh=(np.pi/4,2*np.pi/4))
sobelx_img = abs_sobel_thresh(undist, orient='x', thresh_min=50, thresh_max=100)
sobely_img = abs_sobel_thresh(undist, orient='y', thresh_min=50, thresh_max=100)

cv2.imshow('sobelx', sobelx_img)
cv2.imshow('sobely', sobely_img)
# cv2.imshow('dir_img', dir_img)
cv2.imshow('mag_img', mag_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
combined = np.zeros_like(dir_img)
combined[((sobelx == 255) & (sobely == 255)) | ((mag_img == 255) & (dir_img == 255)) | (hls_img == 255)] = 255
cv2.imwrite('output_images/combinethreshImage.jpg', combined)
cv2.imshow("combine", combined)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
ret, mtx, dist, rvecs, tvecs = calibration()

In [ ]:
#Calculating Distortion Matrix
ret, mtx, dist, rvecs, tvecs = calibration()
original = cv2.imread('camera_cal/calibration1.jpg')
undist = cv2.undistort(original, mtx, dist, None, mtx)
cv2.imwrite('output_images/undistortImage.jpg', undist)
# cv2.imshow('original', original)
# cv2.imshow('undist', undist)
# cv2.waitKey(0)
# cv2.destroyAllWindows()